##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard 이동: TensorFlow의 시각화 도구모음

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tensorboard">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/tensorboard.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     <br>Google Colab에서 실행</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/tensorboard.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     <br>GitHub에서 소스 보기</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"><br>노트북 다운로드</a>
</td>
</table>


[TensorBoard](https://www.tensorflow.org/tensorboard) 는 TensorFlow에서 측정과 시각화를 제공하는 내장 도구이다. 정확도와 손실 같은, 머신러닝 실험의 공통의 기준들은 TensorBoard가 추적하고 보여줄 수 있다. TensorBoard 는 TensorFlow 1과 2의 코드와 호환된다.

TensorFlow 1에서는, `tf.estimator.Estimator` 기본적으로 TensorBoard에 대한 개요를 저장한다. 그에 비해, TensorFlow 2에서는 , 개요는 `tf.keras.callbacks.TensorBoard` <a href="https://keras.io/api/callbacks/" class="external">callback</a> 을 사용해서 저장한다.

이 가이드는 첫째로 TensorFlow 1에서 평가기준을 가지고 어떻게 TensorBoard를 이용하는지와 TensorFlow 2에서 어떻게 동등한 프로세스를 수행할 수 있는지에 대해서 보여준다.

### 설정

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import tempfile
import numpy as np
import datetime
%load_ext tensorboard

In [ ]:
mnist = tf.keras.datasets.mnist # The MNIST dataset.

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### TensorFlow 1: tf.estimator와 TensorBoard

이 TensorFlow 1 예시에서, `tf.estimator.DNNClassifier`을 예시로 들어 설명하는데, MNIST dataset에서 훈련하고 평가하며, 기준들을 보여주기 위해서 TensorBoard를 이용한다:

In [ ]:
%reload_ext tensorboard

feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.1,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%tensorboard --logdir {classifier.model_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF1.png"/> -->

### TensorFlow 2: Keras callback and Model.fit 과 TensorBoard

이 TensorFlow 2 예시에서,`tf.keras.callbacks.TensorBoard` callback와 함께 로그를 생성하고 저장하고, 모델을 훈련시킨다. callback 정확도와 loss per epoch를 추적한다. 그리고  in the `callbacks` 리스트 안에 있는 `Model.fit 에 전달된다.`

In [ ]:
%reload_ext tensorboard

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir,
  histogram_freq=1) # Enable histogram computation with each epoch.

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir {tensorboard_callback.log_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF2.png"/> -->


# 다음 단계

- [Get started](https://www.tensorflow.org/tensorboard/get_started) 가이드에서 TensorBoard에 대한 더 많은 것을 알아보기.
- lower level APIs에 대해서는, [tf.summary migration to TensorFlow 2](https://www.tensorflow.org/tensorboard/migrate) 가이드를 참조.